In [1]:
import pandas as pd
from tqdm import tqdm
from time import sleep
from jqdatasdk import auth, get_query_count, query, get_all_securities, get_industry, get_history_fundamentals, get_valuation, get_all_factors, \
                      finance, balance, income, cash_flow, valuation

In [3]:
auth('19136929902','20021020Liu')
get_query_count()

auth success 


{'total': 1000000, 'spare': 1000000}

In [48]:
auth('15657101368','Ll1753235')
get_query_count()

auth success 


{'total': 1000000, 'spare': 990425}

In [4]:
auth('13156833095','Chen123456')
get_query_count()

auth success 


{'total': 1000000, 'spare': 1000000}

In [4]:
stat_date = '2023q1'
start_date = '1990-01-01'
end_date = '2023-05-01'

In [15]:
# 所有股票数据
df1 = get_all_securities()
df2 = df1.reset_index()
df3 = df2.rename(columns={'index': '股票代码', 'display_name': '中文名称', 'name': '缩写简称', 'start_date': '上市日期', 'end_date': '退市日期', 'type': '类型'})
df3['证券代码'] = df3['股票代码'].apply(lambda x: int(x.split('.')[0]))
df3.to_csv('../../data/jqdata/所有股票数据.csv', index=False, encoding='utf-8-sig')
df3

,股票代码,中文名称,缩写简称,上市日期,退市日期,类型,证券代码
0,000001.XSHE,平安银行,PAYH,1991-04-03,2200-01-01,stock,1
1,000002.XSHE,万科A,WKA,1991-01-29,2200-01-01,stock,2
2,000004.XSHE,ST国华,STGH,1990-12-01,2200-01-01,stock,4
3,000005.XSHE,ST星源,STXY,1990-12-10,2200-01-01,stock,5
4,000006.XSHE,深振业A,SZYA,1992-04-27,2200-01-01,stock,6
...,...,...,...,...,...,...,...
5150,688799.XSHG,华纳药厂,HNYC,2021-07-13,2200-01-01,stock,688799
5151,688800.XSHG,瑞可达,RKD,2021-07-22,2200-01-01,stock,688800
5152,688819.XSHG,天能股份,TNGF,2021-01-18,2200-01-01,stock,688819
5153,688981.XSHG,中芯国际,ZXGJ,2020-07-16,2200-01-01,stock,688981


In [6]:
sea_lst = []
for y in range(1990, 2024):
    for s in ['03-31', '06-30', '09-30', '12-31']:
        sea_lst.append(f'{y}-{s}')
sea_df = pd.DataFrame(sea_lst[:-3], columns=['会计期间'])
stk_df = pd.read_csv('../../data/jqdata/所有股票数据.csv')
stk_df['on'] = 0
sea_df['on'] = 0
header = pd.merge(stk_df, sea_df, on=['on'], how='left')[['证券代码', '会计期间']]
header

,证券代码,会计期间
0,1,1990-03-31
1,1,1990-06-30
2,1,1990-09-30
3,1,1990-12-31
4,1,1991-03-31
...,...,...
685610,689009,2022-03-31
685611,689009,2022-06-30
685612,689009,2022-09-30
685613,689009,2022-12-31


In [7]:
code_list = list(pd.read_csv('../../data/jqdata/所有股票数据.csv')['股票代码'])
print(code_list)

['000001.XSHE', '000002.XSHE', '000004.XSHE', '000005.XSHE', '000006.XSHE', '000007.XSHE', '000008.XSHE', '000009.XSHE', '000010.XSHE', '000011.XSHE', '000012.XSHE', '000014.XSHE', '000016.XSHE', '000017.XSHE', '000018.XSHE', '000019.XSHE', '000020.XSHE', '000021.XSHE', '000022.XSHE', '000023.XSHE', '000024.XSHE', '000025.XSHE', '000026.XSHE', '000027.XSHE', '000028.XSHE', '000029.XSHE', '000030.XSHE', '000031.XSHE', '000032.XSHE', '000033.XSHE', '000034.XSHE', '000035.XSHE', '000036.XSHE', '000037.XSHE', '000038.XSHE', '000039.XSHE', '000040.XSHE', '000042.XSHE', '000043.XSHE', '000045.XSHE', '000046.XSHE', '000048.XSHE', '000049.XSHE', '000050.XSHE', '000055.XSHE', '000056.XSHE', '000058.XSHE', '000059.XSHE', '000060.XSHE', '000061.XSHE', '000062.XSHE', '000063.XSHE', '000065.XSHE', '000066.XSHE', '000068.XSHE', '000069.XSHE', '000070.XSHE', '000078.XSHE', '000088.XSHE', '000089.XSHE', '000090.XSHE', '000096.XSHE', '000099.XSHE', '000100.XSHE', '000150.XSHE', '000151.XSHE', '000153.X

In [48]:
# 资产负债数据
df_list = []
for code in tqdm(code_list):
    while True:
        try:
            df1 = get_history_fundamentals(code, 
                                        [balance.code, balance.pubDate, balance.statDate, balance.cash_equivalents, balance.settlement_provi, balance.lend_capital, balance.trading_assets, balance.bill_receivable, balance.account_receivable, balance.advance_payment, balance.insurance_receivables, balance.reinsurance_receivables, balance.reinsurance_contract_reserves_receivable, balance.interest_receivable, balance.dividend_receivable, balance.other_receivable, balance.bought_sellback_assets, balance.inventories, balance.non_current_asset_in_one_year, balance.other_current_assets, balance.total_current_assets, balance.loan_and_advance, balance.hold_for_sale_assets, balance.hold_to_maturity_investments, balance.longterm_receivable_account, balance.longterm_equity_invest, balance.investment_property, balance.fixed_assets, balance.constru_in_process, balance.construction_materials, balance.fixed_assets_liquidation, balance.biological_assets, balance.oil_gas_assets, balance.intangible_assets, balance.development_expenditure, balance.good_will, balance.long_deferred_expense, balance.deferred_tax_assets, balance.other_non_current_assets, balance.total_non_current_assets, balance.total_assets, balance.shortterm_loan, balance.borrowing_from_centralbank, balance.deposit_in_interbank, balance.borrowing_capital, balance.trading_liability, balance.notes_payable, balance.accounts_payable, balance.advance_peceipts, balance.sold_buyback_secu_proceeds, balance.commission_payable, balance.salaries_payable, balance.taxs_payable, balance.interest_payable, balance.dividend_payable, balance.other_payable, balance.reinsurance_payables, balance.insurance_contract_reserves, balance.proxy_secu_proceeds, balance.receivings_from_vicariously_sold_securities, balance.non_current_liability_in_one_year, balance.other_current_liability, balance.total_current_liability, balance.longterm_loan, balance.bonds_payable, balance.longterm_account_payable, balance.specific_account_payable, balance.estimate_liability, balance.deferred_tax_liability, balance.other_non_current_liability, balance.total_non_current_liability, balance.total_liability, balance.paidin_capital, balance.capital_reserve_fund, balance.treasury_stock, balance.specific_reserves, balance.surplus_reserve_fund, balance.ordinary_risk_reserve_fund, balance.retained_profit, balance.foreign_currency_report_conv_diff, balance.equities_parent_company_owners, balance.minority_interests, balance.total_owner_equities, balance.total_sheet_owner_equities], 
                                        stat_date=stat_date, 
                                        count=1000)
            sleep(0.1)
            break
        except:
            sleep(1)
            continue
    df2 = df1.rename(columns=
                    {'code': '股票代码', 'pubDate': '报告日期', 'statDate': '会计期间', 'cash_equivalents': '货币资金', 'settlement_provi': '结算备付金', 'lend_capital': '拆出资金', 'trading_assets': '交易性金融资产', 'bill_receivable': '应收票据', 'account_receivable': '应收账款', 'advance_payment': '预付款项', 'insurance_receivables': '应收保费', 'reinsurance_receivables': '应收分保账款', 'reinsurance_contract_reserves_receivable': '应收分保合同准备金', 'interest_receivable': '应收利息', 'dividend_receivable': '应收股利', 'other_receivable': '其他应收款', 'bought_sellback_assets': '买入返售金融资产', 'inventories': '存货', 'non_current_asset_in_one_year': '一年内到期的非流动资产', 'other_current_assets': '其他流动资产', 'total_current_assets': '流动资产合计', 'loan_and_advance': '发放委托贷款及垫款', 'hold_for_sale_assets': '可供出售金融资产', 'hold_to_maturity_investments': '持有至到期投资', 'longterm_receivable_account': '长期应收款', 'longterm_equity_invest': '长期股权投资', 'investment_property': '投资性房地产', 'fixed_assets': '固定资产', 'constru_in_process': '在建工程', 'construction_materials': '工程物资', 'fixed_assets_liquidation': '固定资产清理', 'biological_assets': '生产性生物资产', 'oil_gas_assets': '油气资产', 'intangible_assets': '无形资产', 'development_expenditure': '开发支出', 'good_will': '商誉', 'long_deferred_expense': '长期待摊费用', 'deferred_tax_assets': '递延所得税资产', 'other_non_current_assets': '其他非流动资产', 'total_non_current_assets': '非流动资产合计', 'total_assets': '资产总计', 'shortterm_loan': '短期借款', 'borrowing_from_centralbank': '向中央银行借款', 'deposit_in_interbank': '吸收存款及同业存放', 'borrowing_capital': '拆入资金', 'trading_liability': '交易性金融负债', 'notes_payable': '应付票据', 'accounts_payable': '应付账款', 'advance_peceipts': '预收款项', 'sold_buyback_secu_proceeds': '卖出回购金融资产款', 'commission_payable': '应付手续费及佣金', 'salaries_payable': '应付职工薪酬', 'taxs_payable': '应交税费', 'interest_payable': '应付利息', 'dividend_payable': '应付股利', 'other_payable': '其他应付款', 'reinsurance_payables': '应付分保账款', 'insurance_contract_reserves': '保险合同准备金', 'proxy_secu_proceeds': '代理买卖证券款', 'receivings_from_vicariously_sold_securities': '代理承销证券款', 'non_current_liability_in_one_year': '一年内到期的非流动负债', 'other_current_liability': '其他流动负债', 'total_current_liability': '流动负债合计', 'longterm_loan': '长期借款', 'bonds_payable': '应付债券', 'longterm_account_payable': '长期应付款', 'specific_account_payable': '专项应付款', 'estimate_liability': '预计负债', 'deferred_tax_liability': '递延所得税负债', 'other_non_current_liability': '其他非流动负债', 'total_non_current_liability': '非流动负债合计', 'total_liability': '负债合计', 'paidin_capital': '实收资本(或股本)', 'capital_reserve_fund': '资本公积金', 'treasury_stock': '库存股', 'specific_reserves': '专项储备', 'surplus_reserve_fund': '盈余公积金', 'ordinary_risk_reserve_fund': '一般风险准备', 'retained_profit': '未分配利润', 'foreign_currency_report_conv_diff': '外币报表折算差额', 'equities_parent_company_owners': '归属于母公司股东权益合计', 'minority_interests': '少数股东权益', 'total_owner_equities': '股东权益合计', 'total_sheet_owner_equities': '负债和股东权益合计'}
                    )
    df_list.append(df2)

df3 = pd.concat(df_list, axis=0)
df3['证券代码'] = df3['股票代码'].apply(lambda x: int(x.split('.')[0]))
df4 = df3.drop_duplicates(subset=['证券代码', '会计期间'])
df5 = df4.sort_values(by=['证券代码', '会计期间']).reset_index().drop(columns=['index'])
df6 = pd.merge(header, df5, on=['证券代码', '会计期间'], how='left')
df6['查询成功'] = 1
df6['查询成功'][df6['股票代码'].isna()] = 0
df6.to_csv('../../data/jqdata/季_资产负债数据.csv', index=False, encoding='utf-8-sig')
df6

100%|██████████| 5155/5155 [18:22<00:00,  4.67it/s]  
/tmp/ipykernel_3500/1238679516.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['查询成功'][df6['报告日期'].isna()] = 0


,证券代码,会计期间,股票代码,报告日期,货币资金,结算备付金,拆出资金,交易性金融资产,应收票据,应收账款,...,专项储备,盈余公积金,一般风险准备,未分配利润,外币报表折算差额,归属于母公司股东权益合计,少数股东权益,股东权益合计,负债和股东权益合计,查询成功
0,1,1990-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,1990-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,1990-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,1991-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685610,689009,2022-03-31,689009.XSHG,2022-04-30,1.959409e+09,NaN,NaN,1.655134e+08,NaN,7.725841e+08,...,NaN,NaN,NaN,-3.033019e+09,NaN,4.337840e+09,-1813690.77,4.336026e+09,7.670039e+09,1
685611,689009,2022-06-30,689009.XSHG,2022-08-16,2.890017e+09,NaN,NaN,3.601369e+08,NaN,1.163902e+09,...,NaN,NaN,NaN,-2.815914e+09,NaN,4.597553e+09,-923756.83,4.596629e+09,8.988434e+09,1
685612,689009,2022-09-30,689009.XSHG,2022-10-25,3.315614e+09,NaN,NaN,4.630571e+08,NaN,1.185922e+09,...,NaN,NaN,NaN,-2.677845e+09,NaN,4.784433e+09,-1174193.53,4.783259e+09,9.427026e+09,1
685613,689009,2022-12-31,689009.XSHG,2023-03-31,3.080605e+09,NaN,NaN,6.851096e+08,22521616.41,1.162098e+09,...,NaN,NaN,NaN,-2.620912e+09,NaN,4.896933e+09,48686397.46,4.945619e+09,9.392875e+09,1


In [55]:
# 利润数据
df_list = []
for code in tqdm(code_list):
    while True:
        try:
            df1 = get_history_fundamentals(code, 
                                        [income.code, income.pubDate, income.statDate, income.total_operating_revenue, income.operating_revenue, income.interest_income, income.premiums_earned, income.commission_income, income.total_operating_cost, income.operating_cost, income.interest_expense, income.commission_expense, income.refunded_premiums, income.net_pay_insurance_claims, income.withdraw_insurance_contract_reserve, income.policy_dividend_payout, income.reinsurance_cost, income.operating_tax_surcharges, income.sale_expense, income.administration_expense, income.financial_expense, income.asset_impairment_loss, income.fair_value_variable_income, income.investment_income, income.invest_income_associates, income.exchange_income, income.operating_profit, income.non_operating_revenue, income.non_operating_expense, income.disposal_loss_non_current_liability, income.total_profit, income.income_tax_expense, income.net_profit, income.np_parent_company_owners, income.minority_profit, income.basic_eps, income.diluted_eps, income.other_composite_income, income.total_composite_income, income.ci_parent_company_owners, income.ci_minority_owners], 
                                        stat_date=stat_date, 
                                        count=1000)
            sleep(0.1)
            break
        except:
            sleep(1)
            continue
    df2 = df1.rename(columns=
                    {'code': '股票代码', 'pubDate': '报告日期', 'statDate': '会计期间', 'total_operating_revenue': '营业总收入', 'operating_revenue': '营业收入', 'interest_income': '利息收入', 'premiums_earned': '已赚保费', 'commission_income': '手续费及佣金收入', 'total_operating_cost': '营业总成本', 'operating_cost': '营业成本', 'interest_expense': '利息支出', 'commission_expense': '手续费及佣金支出', 'refunded_premiums': '退保金', 'net_pay_insurance_claims': '赔付支出净额', 'withdraw_insurance_contract_reserve': '提取保险合同准备金净额', 'policy_dividend_payout': '保单红利支出', 'reinsurance_cost': '分保费用', 'operating_tax_surcharges': '营业税金及附加', 'sale_expense': '销售费用', 'administration_expense': '管理费用', 'financial_expense': '财务费用', 'asset_impairment_loss': '资产减值损失', 'fair_value_variable_income': '公允价值变动收益', 'investment_income': '投资收益', 'invest_income_associates': '对联营企业和合营企业的投资收益', 'exchange_income': '汇兑收益', 'operating_profit': '营业利润', 'non_operating_revenue': '营业外收入', 'non_operating_expense': '营业外支出', 'disposal_loss_non_current_liability': '非流动资产处置净损失', 'total_profit': '利润总额', 'income_tax_expense': '所得税费用', 'net_profit': '净利润', 'np_parent_company_owners': '归属于母公司股东的净利润', 'minority_profit': '少数股东损益', 'basic_eps': '基本每股收益', 'diluted_eps': '稀释每股收益', 'other_composite_income': '其他综合收益', 'total_composite_income': '综合收益总额', 'ci_parent_company_owners': '归属于母公司所有者的综合收益总额', 'ci_minority_owners': '归属于少数股东的综合收益总额'}
                    )
    df_list.append(df2)

df3 = pd.concat(df_list, axis=0)
df3['证券代码'] = df3['股票代码'].apply(lambda x: int(x.split('.')[0]))
df4 = df3.drop_duplicates(subset=['证券代码', '会计期间'])
df5 = df4.sort_values(by=['证券代码', '会计期间']).reset_index().drop(columns=['index'])
df6 = pd.merge(header, df5, on=['证券代码', '会计期间'], how='left')
df6['查询成功'] = 1
df6['查询成功'][df6['股票代码'].isna()] = 0
df6.to_csv('../../data/jqdata/季_利润数据.csv', index=False, encoding='utf-8-sig')
df6

100%|██████████| 5155/5155 [18:44<00:00,  4.58it/s]
/tmp/ipykernel_3500/1365482699.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['查询成功'][df6['报告日期'].isna()] = 0


,证券代码,会计期间,股票代码,报告日期,营业总收入,营业收入,利息收入,已赚保费,手续费及佣金收入,营业总成本,...,净利润,归属于母公司股东的净利润,少数股东损益,基本每股收益,稀释每股收益,其他综合收益,综合收益总额,归属于母公司所有者的综合收益总额,归属于少数股东的综合收益总额,查询成功
0,1,1990-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,1990-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,1990-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,1991-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685610,689009,2022-03-31,689009.XSHG,2022-04-30,1.917320e+09,1.917320e+09,NaN,NaN,NaN,1.863121e+09,...,3.793527e+07,3.844657e+07,-511300.50,0.5500,0.53,-3794445.32,3.414082e+07,3.465212e+07,-511300.50,1
685611,689009,2022-06-30,689009.XSHG,2022-08-16,2.903166e+09,2.903166e+09,NaN,NaN,NaN,2.573851e+09,...,2.173315e+08,2.171050e+08,226438.78,3.0584,2.83,7034429.90,2.243659e+08,2.241395e+08,226438.78,1
685612,689009,2022-09-30,689009.XSHG,2022-10-25,2.811668e+09,2.811668e+09,NaN,NaN,NaN,2.584274e+09,...,1.378179e+08,1.380683e+08,-250436.70,1.9467,1.79,1043325.15,1.388612e+08,1.391117e+08,-250436.70,1
685613,689009,2022-12-31,689009.XSHG,2023-03-31,2.492164e+09,2.492164e+09,NaN,NaN,NaN,2.453002e+09,...,5.551876e+07,5.693316e+07,-1414409.01,0.8011,0.57,44192860.55,9.971162e+07,1.011260e+08,-1414409.01,1


In [58]:
# 现金流数据
df_list = []
for code in tqdm(code_list):
    while True:
        try:
            df1 = get_history_fundamentals(code, 
                                        [cash_flow.code, cash_flow.pubDate, cash_flow.statDate, cash_flow.goods_sale_and_service_render_cash, cash_flow.net_deposit_increase, cash_flow.net_borrowing_from_central_bank, cash_flow.net_borrowing_from_finance_co, cash_flow.net_original_insurance_cash, cash_flow.net_cash_received_from_reinsurance_business, cash_flow.net_insurer_deposit_investment, cash_flow.net_deal_trading_assets, cash_flow.interest_and_commission_cashin, cash_flow.net_increase_in_placements, cash_flow.net_buyback, cash_flow.tax_levy_refund, cash_flow.other_cashin_related_operate, cash_flow.subtotal_operate_cash_inflow, cash_flow.goods_and_services_cash_paid, cash_flow.net_loan_and_advance_increase, cash_flow.net_deposit_in_cb_and_ib, cash_flow.original_compensation_paid, cash_flow.handling_charges_and_commission, cash_flow.policy_dividend_cash_paid, cash_flow.staff_behalf_paid, cash_flow.tax_payments, cash_flow.other_operate_cash_paid, cash_flow.subtotal_operate_cash_outflow, cash_flow.net_operate_cash_flow, cash_flow.invest_withdrawal_cash, cash_flow.invest_proceeds, cash_flow.fix_intan_other_asset_dispo_cash, cash_flow.net_cash_deal_subcompany, cash_flow.other_cash_from_invest_act, cash_flow.subtotal_invest_cash_inflow, cash_flow.fix_intan_other_asset_acqui_cash, cash_flow.invest_cash_paid, cash_flow.impawned_loan_net_increase, cash_flow.net_cash_from_sub_company, cash_flow.other_cash_to_invest_act, cash_flow.subtotal_invest_cash_outflow, cash_flow.net_invest_cash_flow, cash_flow.cash_from_invest, cash_flow.cash_from_mino_s_invest_sub, cash_flow.cash_from_borrowing, cash_flow.cash_from_bonds_issue, cash_flow.other_finance_act_cash, cash_flow.subtotal_finance_cash_inflow, cash_flow.borrowing_repayment, cash_flow.dividend_interest_payment, cash_flow.proceeds_from_sub_to_mino_s, cash_flow.other_finance_act_payment, cash_flow.subtotal_finance_cash_outflow, cash_flow.net_finance_cash_flow, cash_flow.exchange_rate_change_effect, cash_flow.cash_equivalent_increase, cash_flow.cash_equivalents_at_beginning, cash_flow.cash_and_equivalents_at_end], 
                                        stat_date=stat_date, 
                                        count=1000)
            sleep(0.1)
            break
        except:
            sleep(1)
            continue
    df2 = df1.rename(columns=
                    {'code': '股票代码', 'pubDate': '报告日期', 'statDate': '会计期间', 'goods_sale_and_service_render_cash': '销售商品、提供劳务收到的现金', 'net_deposit_increase': '客户存款和同业存放款项净增加额', 'net_borrowing_from_central_bank': '向中央银行借款净增加额', 'net_borrowing_from_finance_co': '向其他金融机构拆入资金净增加额', 'net_original_insurance_cash': '收到原保险合同保费取得的现金', 'net_cash_received_from_reinsurance_business': '收到再保险业务现金净额', 'net_insurer_deposit_investment': '保户储金及投资款净增加额', 'net_deal_trading_assets': '处置交易性金融资产净增加额', 'interest_and_commission_cashin': '收取利息、手续费及佣金的现金', 'net_increase_in_placements': '拆入资金净增加额', 'net_buyback': '回购业务资金净增加额', 'tax_levy_refund': '收到的税费返还', 'other_cashin_related_operate': '收到其他与经营活动有关的现金', 'subtotal_operate_cash_inflow': '经营活动现金流入小计', 'goods_and_services_cash_paid': '购买商品、接受劳务支付的现金', 'net_loan_and_advance_increase': '客户贷款及垫款净增加额', 'net_deposit_in_cb_and_ib': '存放中央银行和同业款项净增加额', 'original_compensation_paid': '支付原保险合同赔付款项的现金', 'handling_charges_and_commission': '支付利息、手续费及佣金的现金', 'policy_dividend_cash_paid': '支付保单红利的现金', 'staff_behalf_paid': '支付给职工以及为职工支付的现金', 'tax_payments': '支付的各项税费', 'other_operate_cash_paid': '支付其他与经营活动有关的现金', 'subtotal_operate_cash_outflow': '经营活动现金流出小计', 'net_operate_cash_flow': '经营活动产生的现金流量净额', 'invest_withdrawal_cash': '收回投资收到的现金', 'invest_proceeds': '取得投资收益收到的现金', 'fix_intan_other_asset_dispo_cash': '处置固定资产、无形资产和其他长期资产收回的现金净额', 'net_cash_deal_subcompany': '处置子公司及其他营业单位收到的现金净额', 'other_cash_from_invest_act': '收到其他与投资活动有关的现金', 'subtotal_invest_cash_inflow': '投资活动现金流入小计', 'fix_intan_other_asset_acqui_cash': '购建固定资产、无形资产和其他长期资产支付的现金', 'invest_cash_paid': '投资支付的现金', 'impawned_loan_net_increase': '质押贷款净增加额', 'net_cash_from_sub_company': '取得子公司及其他营业单位支付的现金净额', 'other_cash_to_invest_act': '支付其他与投资活动有关的现金', 'subtotal_invest_cash_outflow': '投资活动现金流出小计', 'net_invest_cash_flow': '投资活动产生的现金流量净额', 'cash_from_invest': '吸收投资收到的现金', 'cash_from_mino_s_invest_sub': '子公司吸收少数股东投资收到的现金', 'cash_from_borrowing': '取得借款收到的现金', 'cash_from_bonds_issue': '发行债券收到的现金', 'other_finance_act_cash': '收到其他与筹资活动有关的现金', 'subtotal_finance_cash_inflow': '筹资活动现金流入小计', 'borrowing_repayment': '偿还债务支付的现金', 'dividend_interest_payment': '分配股利、利润或偿付利息支付的现金', 'proceeds_from_sub_to_mino_s': '子公司支付给少数股东的股利、利润', 'other_finance_act_payment': '支付其他与筹资活动有关的现金', 'subtotal_finance_cash_outflow': '筹资活动现金流出小计', 'net_finance_cash_flow': '筹资活动产生的现金流量净额', 'exchange_rate_change_effect': '汇率变动对现金及现金等价物的影响', 'cash_equivalent_increase': '现金及现金等价物净增加额', 'cash_equivalents_at_beginning': '期初现金及现金等价物余额', 'cash_and_equivalents_at_end': '期末现金及现金等价物余额'}
                    )
    df_list.append(df2)

df3 = pd.concat(df_list, axis=0)
df3['证券代码'] = df3['股票代码'].apply(lambda x: int(x.split('.')[0]))
df4 = df3.drop_duplicates(subset=['证券代码', '会计期间'])
df5 = df4.sort_values(by=['证券代码', '会计期间']).reset_index().drop(columns=['index'])
df6 = pd.merge(header, df5, on=['证券代码', '会计期间'], how='left')
df6['查询成功'] = 1
df6['查询成功'][df6['股票代码'].isna()] = 0
df6.to_csv('../../data/jqdata/季_现金流数据.csv', index=False, encoding='utf-8-sig')
df6

100%|██████████| 5155/5155 [16:29<00:00,  5.21it/s]
/tmp/ipykernel_3500/3635249507.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['查询成功'][df6['报告日期'].isna()] = 0


,证券代码,会计期间,股票代码,报告日期,销售商品、提供劳务收到的现金,客户存款和同业存放款项净增加额,向中央银行借款净增加额,向其他金融机构拆入资金净增加额,收到原保险合同保费取得的现金,收到再保险业务现金净额,...,分配股利、利润或偿付利息支付的现金,子公司支付给少数股东的股利、利润,支付其他与筹资活动有关的现金,筹资活动现金流出小计,筹资活动产生的现金流量净额,汇率变动对现金及现金等价物的影响,现金及现金等价物净增加额,期初现金及现金等价物余额,期末现金及现金等价物余额,查询成功
0,1,1990-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,1990-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,1990-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,1991-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685610,689009,2022-03-31,689009.XSHG,2022-04-30,2.196263e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4485193.73,4.485194e+06,-3.985194e+06,-5269825.39,2.568291e+08,1.685944e+09,1.942773e+09,1
685611,689009,2022-06-30,689009.XSHG,2022-08-16,2.759823e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5506988.79,5.506989e+06,1.417070e+08,34516524.56,8.514537e+08,1.942773e+09,2.794227e+09,1
685612,689009,2022-09-30,689009.XSHG,2022-10-25,3.422816e+09,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1336192.41,1.412535e+08,-1.371789e+08,35471971.02,3.411207e+08,2.794227e+09,3.135347e+09,1
685613,689009,2022-12-31,689009.XSHG,2023-03-31,2.801277e+09,NaN,NaN,NaN,NaN,NaN,...,-22914.52,NaN,7103917.22,7.081765e+06,4.921765e+07,-23393006.44,-4.312723e+08,3.135347e+09,2.704075e+09,1


In [49]:
# 市值数据

sea_lst = []
for y in range(2005, 2024):
    for s in ['03-31', '06-30', '09-30', '12-31']:
        sea_lst.append(f'{y}-{s}')
sea_lst = sea_lst[:-3]

df_list = []
for season in tqdm(sea_lst):
    for code in (code_list[:int(len(code_list)/2)], code_list[int(len(code_list)/2):]):
        while True:
            try:
                df1 = get_valuation(code, 
                                    fields=['code', 'day', 'capitalization', 'circulating_cap', 'market_cap', 'circulating_market_cap', 'pe_ratio', 'pb_ratio', 'ps_ratio', 'pcf_ratio'], 
                                    end_date=season, 
                                    count=1)
                sleep(0.1)
                break
            except:
                sleep(1)
                continue
        df2 = df1.rename(columns=
                        {'code': '股票代码', 'day': '报告日期', 'capitalization': '总股本', 'circulating_cap': '流通股本', 'market_cap': '总市值', 'circulating_market_cap': '流通市值', 'pe_ratio': '市盈率(TTM)', 'pb_ratio': '市净率', 'ps_ratio': '市销率(TTM)', 'pcf_ratio': '市现率(TTM)'}
                        )
        df_list.append(df2)

df3 = pd.concat(df_list, axis=0)
df3['报告日期'] = df3['报告日期'].astype('str')
df4 = df3[(df3['报告日期'].apply(lambda x: x[5:7]).isin(['03', '06', '09', '12'])) & (df3['报告日期'].apply(lambda x: int(x[-2:])) > 25)]
df4['证券代码'] = df4['股票代码'].apply(lambda x: int(x.split('.')[0]))
df4['会计期间'] = df4['报告日期'].apply(lambda x: x[:-2] + '30' if x[5:7] in ['06', '09'] else x[:-2] + '31')
df4['总股本'] = 10000*df4['总股本']
df4['流通股本'] = 10000*df4['流通股本']
df4['总市值'] = 100000000*df4['总市值']
df4['流通市值'] = 100000000*df4['流通市值']
df5 = df4.drop_duplicates(subset=['证券代码', '会计期间'])
df6 = df5.sort_values(by=['证券代码', '会计期间']).reset_index().drop(columns=['index'])
df7 = pd.merge(header, df6, on=['证券代码', '会计期间'], how='left')
df7['查询成功'] = 1
df7['查询成功'][df7['股票代码'].isna()] = 0
df7.to_csv('../../data/jqdata/季_市值数据.csv', index=False, encoding='utf-8-sig')
df7

100%|██████████| 73/73 [00:49<00:00,  1.47it/s]
/tmp/ipykernel_3546/2345380852.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df7['查询成功'][df7['股票代码'].isna()] = 0


,证券代码,会计期间,股票代码,报告日期,总股本,流通股本,总市值,流通市值,市盈率(TTM),市净率,市销率(TTM),市现率(TTM),查询成功
0,1,1990-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,1990-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,1,1990-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1,1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1,1991-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
685610,689009,2022-03-31,689009.XSHG,2022-03-31,707887188.0,508884062.0,3.099838e+10,2.228403e+10,74.7157,7.3053,3.3965,195.0657,1
685611,689009,2022-06-30,689009.XSHG,2022-06-30,711504297.0,508884062.0,3.166194e+10,2.264534e+10,74.7382,7.2990,3.4101,6185.7881,1
685612,689009,2022-09-30,689009.XSHG,2022-09-30,712983125.0,508884062.0,2.731438e+10,1.949535e+10,66.2633,5.9411,2.9536,97.9056,1
685613,689009,2022-12-31,689009.XSHG,2022-12-30,712983125.0,511700430.0,2.173886e+10,1.560175e+10,52.2658,4.5437,2.2702,14.7436,1


In [45]:
# 上市公司基本信息

df_list = []
for code in tqdm(code_list):
    while True:
        try:
            df1 = finance.run_query(query(finance.STK_COMPANY_INFO).filter(finance.STK_COMPANY_INFO.code == code))
            sleep(0.1)
            break
        except:
            sleep(1)
            continue
    df_list.append(df1)

df2 = pd.concat(df_list)
df2 = df2.rename(columns=
                 {'company_id': '公司ID', 'code': '股票代码', 'full_name': '公司名称', 'short_name': '公司简称', 'a_code': 'A股股票代码', 'b_code': 'B股股票代码', 'h_code': 'H股股票代码', 'fullname_en': '英文名称', 'shortname_en': '英文简称', 'legal_representative': '法人代表', 'register_location': '注册地址', 'office_address': '办公地址', 'zipcode': '邮政编码', 'register_capital': '注册资金', 'currency_id': '货币编码', 'currency': '货币名称', 'establish_date': '成立日期', 'website': '机构网址', 'email': '电子信箱', 'contact_number': '联系电话', 'fax_number': '联系传真', 'main_business': '主营业务', 'business_scope': '经营范围', 'description': '机构简介', 'tax_number': '税务登记号', 'license_number': '法人营业执照号', 'pub_newspaper': '指定信息披露报刊', 'pub_website': '指定信息披露网站', 'secretary': '董事会秘书', 'secretary_number': '董秘联系电话', 'secretary_fax': '董秘联系传真', 'secretary_email': '董秘电子邮箱', 'security_representative': '证券事务代表', 'province_id': '所属省份编码', 'province': '所属省份', 'city_id': '所属城市编码', 'city': '所属城市', 'industry_id': '行业编码', 'industry_1': '行业一级分类', 'industry_2': '行业二级分类', 'cpafirm': '会计师事务所', 'lawfirm': '律师事务所', 'ceo': '总经理', 'comments': '备注'}
                 )
df2['证券代码'] = df2['股票代码'].apply(lambda x: int(x.split('.')[0]))
df2 = df2.drop(columns=['id'])
stk_df = pd.read_csv('../../data/jqdata/所有股票数据.csv')[['证券代码']]
df3 = pd.merge(stk_df, df2, on=['证券代码'], how='left')
df3['查询成功'] = 1
df3['查询成功'][df3['股票代码'].isna()] = 0
df3.to_csv('../../data/jqdata/上市公司基本信息.csv', index=False, encoding='utf-8-sig')
df3

/tmp/ipykernel_43575/2576041965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['查询成功'][df3['股票代码'].isna()] = 0


,证券代码,公司ID,股票代码,公司名称,公司简称,A股股票代码,B股股票代码,H股股票代码,英文名称,英文简称,...,所属城市编码,所属城市,行业编码,行业一级分类,行业二级分类,会计师事务所,律师事务所,总经理,备注,查询成功
0,1,430000001,000001.XSHE,平安银行股份有限公司,平安银行,000001,None,None,"Ping An Bank Co., Ltd.",Ping An Bank,...,440300,深圳市,J66,金融业,货币金融服务,安永华明会计师事务所(特殊普通合伙),竞天公诚律师事务所,胡跃飞,深发展的核心竞争力更在于其基于市场需求基础上的雄厚的创新实力。迄今深发展已推出双周供、存抵贷...,1
1,2,430000002,000002.XSHE,万科企业股份有限公司,万科,000002,200002,02202,"China Vanke Co., Ltd.",Vanke,...,440300,深圳市,K70,房地产业,房地产业,毕马威华振会计师事务所(特殊普通合伙),信达律师事务所,祝九胜,公司为国内规模最大的地产发展商，连续三年被国务院发展研究中心企业所、清华大学房地产研究所和中...,1
2,4,430000004,000004.XSHE,深圳国华网安科技股份有限公司,国华网安,000004,None,None,Shenzhen GuoHua Network Security Technology Co...,GH-Tech,...,440300,深圳市,I65,信息传输、软件和信息技术服务业,软件和信息技术服务业,中兴财光华会计师事务所(特殊普通合伙),国浩律师集团（深圳）事务所,黄翔,国农科技以农业科技园、房地产、生物制药、高科技软件为产业核心业务。国农科技于2005年11月...,1
3,5,430000005,000005.XSHE,深圳世纪星源股份有限公司,世纪星源,000005,None,None,Shenzhen Fountain Corporation,Fountain,...,440300,深圳市,N77,水利、环境和公共设施管理业,生态保护和环境治理业,中审亚太会计师事务所(特殊普通合伙),信利律师事务所,郑列列,"公司的房地产项目近年增长十分迅猛，即将开工的福华项目在深圳南山区拥有42,651平方米工业用...",1
4,6,430000006,000006.XSHE,深圳市振业(集团)股份有限公司,深振业,000006,None,None,"Shenzhen Zhenye (Group) Co., Ltd.",Shenzhen Zhenye,...,440300,深圳市,K70,房地产业,房地产业,天职国际会计师事务所(特殊普通合伙),广东海埠律师事务所,李伟,公司作为本地著名房产开发商，在南部沿海区域垄断优势明显。相对公司市值，公司的土地储备量显得十...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5150,688799,300033718,688799.XSHG,湖南华纳大药厂股份有限公司,华纳药厂,688799,None,None,"Hunan Warrant Pharmaceutical Co., Ltd.",Warrant Pharmaceutical,...,430100,长沙市,C27,制造业,医药制造业,天健会计师事务所(特殊普通合伙),北京市康达律师事务所,黄本东,None,1
5151,688800,440831274,688800.XSHG,苏州瑞可达连接系统股份有限公司,瑞可达,688800,None,None,"Suzhou Recodeal Interconnect System Co., Ltd.",Recodeal,...,320500,苏州市,C39,制造业,计算机、通信和其他电子设备制造业,容诚会计师事务所(特殊普通合伙),国浩律师(南京)事务所,张杰,None,1
5152,688819,300041888,688819.XSHG,天能电池集团股份有限公司,天能股份,688819,None,None,"Tianneng Battery Group Co., Ltd.",Tianneng,...,330500,湖州市,C38,制造业,电气机械和器材制造业,中汇会计师事务所(特殊普通合伙),北京市中伦律师事务所,杨建芬,None,1
5153,688981,410000981,688981.XSHG,中芯国际集成电路制造有限公司,中芯国际,688981,None,00981,Semiconductor Manufacturing International Corp...,SMIC,...,310100,上海市,C39,制造业,计算机、通信和其他电子设备制造业,普华永道中天会计师事务所(特殊普通合伙),上海市锦天城律师事务所,None,None,1


In [29]:
# 上市公司员工情况

df_list = []
for code in tqdm(code_list):
    while True:
        try:
            df1 = finance.run_query(query(finance.STK_EMPLOYEE_INFO).filter(finance.STK_EMPLOYEE_INFO.code == code))
            sleep(0.1)
            break
        except:
            sleep(1)
            continue
    df_list.append(df1)

df2 = pd.concat(df_list)
df2 = df2.rename(columns=
                 {'company_id': '公司ID', 'code': '股票代码', 'name': '证券名称', 'end_date': '会计期间', 'pub_date': '公告日期', 'employee': '在职员工总数', 'retirement': '离退休人员', 'graduate_rate': '研究生以上人员比例', 'college_rate': '大学专科以上人员比例', 'middle_rate': '中专及以下人员比例'}
                 )
df2['证券代码'] = df2['股票代码'].apply(lambda x: int(x.split('.')[0]))
df2 = df2.drop(columns=['id'])
df3 = pd.merge(header[header['会计期间'].apply(lambda x: True if '12-31' in x else False)], df2, on=['证券代码', '会计期间'], how='left')
df3.to_csv('../../data/jqdata/年_上市公司员工情况.csv', index=False, encoding='utf-8-sig')
df3

,证券代码,会计期间,公司ID,股票代码,证券名称,公告日期,在职员工总数,离退休人员,研究生以上人员比例,大学专科以上人员比例,中专及以下人员比例
0,1,1990-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1991-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1992-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1993-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1994-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
170110,689009,2018-12-31,300039199.0,689009.XSHG,九号公司,2019-04-17,1488.0,NaN,NaN,61.6263,29.7043
170111,689009,2019-12-31,300039199.0,689009.XSHG,九号公司,2020-09-30,2383.0,NaN,NaN,65.5476,25.3042
170112,689009,2020-12-31,300039199.0,689009.XSHG,九号公司,2021-04-16,2654.0,NaN,NaN,67.1439,24.1145
170113,689009,2021-12-31,300039199.0,689009.XSHG,九号公司,2022-04-26,3523.0,NaN,NaN,66.9884,24.9219


In [56]:
# 上市公司股本变动

df_list = []
for code in tqdm(code_list):
    while True:
        try:
            df1 = finance.run_query(query(finance.STK_CAPITAL_CHANGE).filter(finance.STK_CAPITAL_CHANGE.code == code))
            sleep(0.1)
            break
        except:
            sleep(1)
            continue
    df_list.append(df1)

df2 = pd.concat(df_list)
df2 = df2.rename(columns=
                 {'company_id': '公司ID', 'company_name': '公司名称', 'code': '股票代码', 'change_date': '变动日期', 'pub_date': '公告日期', 'change_reason_id': '变动原因编码', 'change_reason': '变动原因', 'share_total': '总股本', 'share_non_trade': '未流通股份', 'share_start': '发起人股份', 'share_nation': '国家持股', 'share_nation_legal': '国有法人持股', 'share_instate_legal': '境内法人持股', 'share_outstate_legal': '境外法人持股', 'share_natural': '自然人持股', 'share_raised': '募集法人股', 'share_inside': '内部职工股', 'share_convert': '转配股', 'share_perferred': '优先股', 'share_other_nontrade': '其他未流通股', 'share_limited': '流通受限股份', 'share_legal_issue': '配售法人股', 'share_strategic_investor': '战略投资者持股', 'share_fund': '证券投资基金持股', 'share_normal_legal': '一般法人持股', 'share_other_limited': '其他流通受限股份', 'share_nation_limited': '国家持股(受限)', 'share_nation_legal_limited': '国有法人持股(受限)', 'other_instate_limited': '其他内资持股(受限)', 'legal_of_other_instate_limited': '其他内资持股(受限)中的境内法人持股', 'natural_of_other_instate_limited': '其他内资持股(受限)中的境内自然人持股', 'outstate_limited': '外资持股(受限)', 'legal_of_outstate_limited': '外资持股(受限)中的境外法人持股', 'natural_of_outstate_limited': '外资持股(受限)境外自然人持股', 'share_trade_total': '已流通股份', 'share_rmb': '人民币普通股', 'share_b': '境内上市外资股(B股)', 'share_b_limited': '限售B股', 'share_h': '境外上市外资股(H股)', 'share_h_limited': '限售H股', 'share_management': '高管股', 'share_management_limited': '限售高管股', 'share_other_trade': '其他流通股', 'control_shareholder_limited': '控股股东、实际控制人(受限)', 'core_employee_limited': '核心员工(受限)', 'individual_fund_limited': '个人或基金(受限)', 'other_legal_limited': '其他法人(受限)', 'other_limited': '其他(受限)'}
                 )
df2['证券代码'] = df2['股票代码'].apply(lambda x: int(x.split('.')[0]))
df2[['发起人股份', '已流通股份', '未流通股份', '总股本', '国家持股', '国有法人持股', '境内法人持股', '境外法人持股', '自然人持股', '募集法人股', '内部职工股', '转配股', '优先股', '其他未流通股', '流通受限股份', '配售法人股', '战略投资者持股', '证券投资基金持股', '一般法人持股', '其他流通受限股份', '国家持股(受限)', '国有法人持股(受限)', '其他内资持股(受限)', '其他内资持股(受限)中的境内法人持股', '其他内资持股(受限)中的境内自然人持股', '外资持股(受限)', '外资持股(受限)中的境外法人持股', '外资持股(受限)境外自然人持股', '人民币普通股', '境内上市外资股(B股)', '限售B股', '境外上市外资股(H股)', '限售H股', '高管股', '限售高管股', '其他流通股', '控股股东、实际控制人(受限)', '核心员工(受限)', '个人或基金(受限)', '其他法人(受限)', '其他(受限)']] = \
    10000*df2[['发起人股份', '已流通股份', '未流通股份', '总股本', '国家持股', '国有法人持股', '境内法人持股', '境外法人持股', '自然人持股', '募集法人股', '内部职工股', '转配股', '优先股', '其他未流通股', '流通受限股份', '配售法人股', '战略投资者持股', '证券投资基金持股', '一般法人持股', '其他流通受限股份', '国家持股(受限)', '国有法人持股(受限)', '其他内资持股(受限)', '其他内资持股(受限)中的境内法人持股', '其他内资持股(受限)中的境内自然人持股', '外资持股(受限)', '外资持股(受限)中的境外法人持股', '外资持股(受限)境外自然人持股', '人民币普通股', '境内上市外资股(B股)', '限售B股', '境外上市外资股(H股)', '限售H股', '高管股', '限售高管股', '其他流通股', '控股股东、实际控制人(受限)', '核心员工(受限)', '个人或基金(受限)', '其他法人(受限)', '其他(受限)']]
df2 = df2.drop(columns=['id'])
df3 = df2.sort_values(by=['证券代码', '变动日期']).reset_index().drop(columns=['index'])
df3.to_csv('../../data/jqdata/日_上市公司股本变动.csv', index=False, encoding='utf-8-sig')
df3

,公司ID,公司名称,股票代码,变动日期,公告日期,变动原因编码,变动原因,总股本,未流通股份,发起人股份,...,限售H股,高管股,限售高管股,其他流通股,控股股东、实际控制人(受限),核心员工(受限),个人或基金(受限),其他法人(受限),其他(受限),证券代码
0,430000001,深圳发展银行股份有限公司,000001.XSHE,1994-07-11,1994-07-11,NaN,"送股,转增",4.310686e+08,133697004.0,133697004.0,...,NaN,300155.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
1,430000001,深圳发展银行股份有限公司,000001.XSHE,1994-08-22,1994-07-11,306005.0,配股上市,4.310686e+08,133697004.0,133697004.0,...,NaN,300155.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
2,430000001,深圳发展银行股份有限公司,000001.XSHE,1995-09-25,1995-09-25,NaN,"送股,转增",5.172824e+08,160070886.0,160070886.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
3,430000001,深圳发展银行股份有限公司,000001.XSHE,1997-08-25,1997-08-25,NaN,"送股,转增",1.551847e+09,480212658.0,480212658.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
4,430000001,深圳发展银行股份有限公司,000001.XSHE,2000-12-08,2000-11-06,306005.0,配股上市,1.945822e+09,552697385.0,125390017.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173652,300039199,九号有限公司,689009.XSHG,2022-06-30,2022-08-16,306019.0,定期报告,7.115043e+07,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,689009
173653,300039199,九号有限公司,689009.XSHG,2022-09-28,2022-09-30,306034.0,期权行权,7.129831e+07,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,689009
173654,300039199,九号有限公司,689009.XSHG,2022-10-31,2022-10-25,306026.0,限售股份上市,7.129831e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,689009
173655,300039199,九号有限公司,689009.XSHG,2022-12-31,2023-03-31,306019.0,定期报告,7.129831e+07,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,689009
